### with this script you can add attributes from raster or polygons to points 

In [55]:
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt

import geopandas
import rasterio
import matplotlib.pyplot as plt
from shapely.geometry import Point

import ee
import geemap
import pandas as pd

In [56]:
# Initialize Earth Engine
ee.Initialize()

##### read points

In [57]:
#points = pd.read_csv('/home/sepal-user/module_results/esbae/DRC/DRC_all_ceo_2015_2022_2023_clean.csv', delimiter=',', low_memory=False)
points = pd.read_csv('/home/sepal-user/module_results/esbae/GAB/GAB_all_ceo_2015_2022_2023_clean.csv', delimiter=',', low_memory=False)
#points = pd.read_csv('/home/sepal-user/module_results/esbae/CMR/CMR_all_ceo_2015_2022_2023_clean.csv',delimiter=',', low_memory=False)
#points = pd.read_csv('/home/sepal-user/module_results/esbae/EQG/EQG_all_ceo_2015_2022_2023_clean.csv',delimiter=',', low_memory=False)
#points = pd.read_csv('/home/sepal-user/module_results/esbae/EQG/EQG_all_ceo_2015_2022_2023_prov.csv',delimiter=',', low_memory=False)
#points = pd.read_csv('/home/sepal-user/module_results/esbae/CAR/CAR_all_ceo_2015_2022_2023_clean.csv',delimiter=',', low_memory=False)
#read GEE points
#points = ee.FeatureCollection('users/faocongo/sbae/sbae_hex16_cmr')

In [58]:
len(points)

10329

In [59]:
all_columns = points.columns.tolist()

# Display the list of column names
print(all_columns)

['UID', 'ID', 'plotid', 'point_id', 'sampleid', 'lon', 'lat', 'sample_geom', 'Ref_FNF_2015', 'Ref_Regeneration', 'Ref_Change_1522', 'Ref_Change_Type_1522', 'ArtFor', 'ArtMine', 'InfraR', 'Urb', 'IndFor', 'Other', 'Other_Desc', 'IndMine', 'IndAg', 'ArtAg', 'Ref_Year_1522', 'Ref_NFtype_2015', 'Ref_Ftype_2015', 'collection', 'interpreted', 'Ref_LCover_2015', 'Ref_Change_Year_1522', 'Ref_Change_LCover_1522', 'Def2016', 'Def2017', 'Def2018', 'Def2019', 'Def2020', 'Def2021', 'Def2022', 'Deg2016', 'Deg2017', 'Deg2018', 'Deg2019', 'Deg2020', 'Deg2021', 'Deg2022', 'Defall', 'Degall', 'Stable', 'NF', 'DensFor', 'DensDryFor', 'SecFor', 'DryOpenFor', 'Mangrove', 'Swamp', 'Gallery', 'Plantation', 'Woodland', 'Shrubland', 'Grassland', 'Aquatic', 'Bare', 'Cultivated', 'Builtup', 'Water', 'DensFor_Def', 'DensDryFor_Def', 'SecFor_Def', 'DryOpenFor_Def', 'Mangrove_Def', 'Swamp_Def', 'Gallery_Def', 'Plantation_Def', 'Woodland_Def', 'DensFor_Deg', 'DensDryFor_Deg', 'SecFor_Deg', 'DryOpenFor_Deg', 'Mangrov

In [60]:
#drop any columns if needed
#esbae = esbae.drop('index_right', axis=1)

In [61]:
#enter Lat and Lon columns
LATcol = 'lat'
LONcol = 'lon'

In [62]:
gdf = geopandas.GeoDataFrame(
    points, geometry=geopandas.points_from_xy(points[LONcol], points[LATcol]), crs="EPSG:4326")

#### add attributes from local shapefile

In [75]:
#poly = "/home/sepal-user/data/admin/RDC_Province_26.shp"
#poly = "/home/sepal-user/data/admin/RCA_adm1_itos.shp"
#poly = "/home/sepal-user/data/admin/cmr_admbnda_adm1_inc_20180104.shp"
#poly = "/home/sepal-user/data/admin/geoBoundaries-GNQ-ADM1.shp"
poly = "/home/sepal-user/data/admin/geoBoundaries-GAB-ADM1.shp"
# Read file using gpd.read_file()
poly_shp = gpd.read_file(poly)
poly_shp.head()

,shapeName,shapeISO,shapeID,shapeGroup,shapeType,shapeName1,geometry
0,Woleu-Ntem,GA-9,98808126B17543062715179,GAB,ADM1,Woleu-Ntem,"POLYGON ((13.15133 1.26335, 12.85648 1.25701, ..."
1,Estuaire,GA-1,98808126B6174087794558,GAB,ADM1,Estuaire,"POLYGON ((10.4189 0.98965, 10.42187 0.82788, 1..."
2,Moyen-Ogooué,GA-3,98808126B18014785017467,GAB,ADM1,Moyen-Ogooue,"POLYGON ((10.83434 0.22485, 11.03868 0.22511, ..."
3,Ogooué-Lolo,GA-7,98808126B26189030995048,GAB,ADM1,Ogooue-Lolo,"POLYGON ((13.28884 0.0763, 13.28682 0.02301, 1..."
4,Haut-Ogooué,GA-2,98808126B76399587683442,GAB,ADM1,Haut-Ogooue,"POLYGON ((12.77941 -1.86166, 12.78632 -1.83539..."


In [76]:
#admin_name = 'ADM1_FR'
admin_name = 'shapeName1'
#admin_name = 'admin1Name'
#admin_name = 'NOM'
#new_name = 'Prefecture'
new_name = 'Province'

In [77]:
poly_crs = poly_shp.crs
print("Current CRS:", poly_crs)

Current CRS: EPSG:4326


In [78]:
#poly_shp = poly_shp.to_crs(gdf.crs)

In [79]:
# Reproject both GeoDataFrames to the same projected CRS (e.g., EPSG:3395 for World Mercator)
gdf = gdf.to_crs("EPSG:3395")
poly_shp = poly_shp.to_crs("EPSG:3395")

# Step 1: Perform the spatial join for intersecting points
joined_data_intersecting = gpd.sjoin(gdf, poly_shp[[admin_name, 'geometry']], how="left", predicate="within", lsuffix='left', rsuffix='right')

# Drop the 'index_right' column if it exists, to avoid conflicts
if 'index_right' in joined_data_intersecting.columns:
    joined_data_intersecting = joined_data_intersecting.drop(columns='index_right')

# Step 2: Identify points without an intersection (NaN values in the Admin_Name column)
no_intersection_points = joined_data_intersecting[joined_data_intersecting[admin_name].isna()]

# Step 3: Perform the nearest spatial join for points without an intersection
nearest_join = gpd.sjoin_nearest(no_intersection_points.drop(columns=admin_name), poly_shp[[admin_name, 'geometry']], how="left", distance_col="distance_to_polygon")

# Drop the 'index_right' column if it exists in the nearest join result
if 'index_right' in nearest_join.columns:
    nearest_join = nearest_join.drop(columns='index_right')

# Step 4: Combine intersecting and nearest joined data into one GeoDataFrame
# Retain only the specified `admin_name` column from `poly_shp`
joined_data_combined = pd.concat([
    joined_data_intersecting.dropna(subset=[admin_name]),
    nearest_join
])

# Select only the columns from `gdf` plus `admin_name`
columns_to_keep = list(gdf.columns) + [admin_name]
joined_data = joined_data_combined[columns_to_keep]

# Display the resulting GeoDataFrame
joined_data.head()

,UID,ID,plotid,point_id,sampleid,lon,lat,sample_geom,Ref_FNF_2015,Ref_Regeneration,...,Ref_LCover_2022,Ref_Change_Year_2023,Ref_Change_LCover_2023,Def2023,Deg2023,interpreted_qc,Ref_Change_Type_1523,Ref_Change_Year_1523,geometry,shapeName1
1,1,1,187092,187092,187092,14,1,POINT(13.994761 0.869878),1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Stable,Stable,POINT (1557889.668 96189.875),Ogooue-Ivindo
2,2,2,185303,185303,185303,13,-0,POINT(13.254088 -0.492938),1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Stable,Stable,POINT (1475438.327 -54506.944),Ogooue-Lolo
3,3,3,185899,185899,185899,14,1,POINT(14.175518 1.275546),1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Stable,Stable,POINT (1578011.445 141054.383),Ogooue-Ivindo
4,4,4,161928,161928,161928,12,-1,POINT(12.303429 -1.380904),1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Stable,Stable,POINT (1369611.451 -152707.442),Ogooue-Lolo
5,5,5,173884,173884,173884,13,-1,POINT(12.853122 -0.772983),1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Stable,Stable,POINT (1430802.996 -85474.663),Ogooue-Lolo


In [80]:
nearest_join.head()

,UID,ID,plotid,point_id,sampleid,lon,lat,sample_geom,Ref_FNF_2015,Ref_Regeneration,...,Ref_Change_Year_2023,Ref_Change_LCover_2023,Def2023,Deg2023,interpreted_qc,Ref_Change_Type_1523,Ref_Change_Year_1523,geometry,shapeName1,distance_to_polygon
0,0,0,157,157,157,10,1,POINT(9.56202 0.937144),1,NaN,...,NaN,NaN,NaN,NaN,NaN,Stable,Stable,POINT (1064439.197 103628.702),Estuaire,965
120,120,120,157707,157707,157707,12,-2,POINT(11.698154 -2.402482),1,NaN,...,NaN,NaN,NaN,NaN,NaN,Stable,Stable,POINT (1302232.547 -265731.63),Ngounie,4153
122,122,122,156677,156677,156677,12,-2,POINT(11.663324 -2.431369),1,NaN,...,NaN,NaN,NaN,NaN,NaN,Stable,Stable,POINT (1298355.289 -268928.672),Ngounie,9155
270,270,270,225142,225142,225142,14,-2,POINT(14.417681 -1.912686),1,NaN,...,NaN,NaN,NaN,NaN,NaN,Stable,Stable,POINT (1604968.907 -211533.688),Haut-Ogooue,3765
285,285,285,65607,65607,65607,10,-3,POINT(9.845223 -2.614749),1,NaN,...,NaN,NaN,NaN,NaN,NaN,Stable,Stable,POINT (1095965.211 -289225.73),Ogooue-Maritime,2494


In [81]:
# Perform the spatial join using the predicate parameter
#joined_data = gpd.sjoin(gdf, poly_shp, how="left", predicate="within", lsuffix='left', rsuffix='right')
# 'joined_data' now contains the points with additional information from the polygons
# Print or explore the resulting GeoDataFrame
#joined_data.head()

In [82]:
joined_data[admin_name].value_counts(dropna=False)

shapeName1
Haut-Ogooue        1816
Ngounie            1510
Ogooue-Ivindo      1275
Ogooue-Maritime    1128
Woleu-Ntem         1084
Nyanga             1022
Estuaire            897
Moyen-Ogooue        810
Ogooue-Lolo         787
Name: count, dtype: int64

In [83]:
province_pts = pd.pivot_table(joined_data,values='plotid',index=[admin_name],columns=['collection'],aggfunc="count",margins=True,
                             margins_name='Total',dropna=False)
province_pts

collection,coll_1522_1,coll_1522_2,coll_1522_3,coll_1522_4,coll_1522_5,coll_2023_1,coll_2023_2,coll_2023_3,Total
shapeName1,,,,,,,,,
Estuaire,41,282,228,NaN,164,50,84,48,897
Haut-Ogooue,62,707,NaN,3,634,67,170,173,1816
Moyen-Ogooue,41,249,3,2,354,45,88,28,810
Ngounie,66,510,NaN,7,654,69,155,49,1510
Nyanga,45,360,NaN,NaN,340,53,138,86,1022
Ogooue-Ivindo,80,400,NaN,10,595,72,81,37,1275
Ogooue-Lolo,58,262,NaN,8,378,40,28,13,787
Ogooue-Maritime,38,412,118,4,338,47,121,50,1128
Woleu-Ntem,68,342,NaN,11,513,54,68,28,1084


In [84]:
# Remove rows where the 'Province' column has NaN values
joined_data = joined_data.dropna(subset=[admin_name])

In [85]:
poly_shp = poly_shp.to_crs("EPSG:3395")

In [86]:
#calculate area of poly
poly_shp['area_sqm'] = poly_shp.geometry.area

In [87]:
poly_shp['area_ha'] = poly_shp['area_sqm'] / 10000

In [88]:
pd.set_option('display.float_format', lambda x: '%.0f' % x)

# Display the GeoDataFrame with the new area columns
print(poly_shp[[admin_name, 'area_ha']])

        shapeName1  area_ha
0       Woleu-Ntem  3674113
1         Estuaire  1965458
2     Moyen-Ogooue  1797702
3      Ogooue-Lolo  2932509
4      Haut-Ogooue  3377135
5          Ngounie  3752265
6  Ogooue-Maritime  2227467
7           Nyanga  1956576
8    Ogooue-Ivindo  4243471


In [89]:
# Rename columns
column_mapping = {
   admin_name: new_name
#    'NAME_2': 'Territoire',
#    'Unnamed: 0':'Index'
}

# Use the rename() method to rename columns
joined_data.rename(columns=column_mapping, inplace=True)

In [90]:
list_columns = joined_data.columns.tolist()

# Display the list of column names
print(list_columns)

['UID', 'ID', 'plotid', 'point_id', 'sampleid', 'lon', 'lat', 'sample_geom', 'Ref_FNF_2015', 'Ref_Regeneration', 'Ref_Change_1522', 'Ref_Change_Type_1522', 'ArtFor', 'ArtMine', 'InfraR', 'Urb', 'IndFor', 'Other', 'Other_Desc', 'IndMine', 'IndAg', 'ArtAg', 'Ref_Year_1522', 'Ref_NFtype_2015', 'Ref_Ftype_2015', 'collection', 'interpreted', 'Ref_LCover_2015', 'Ref_Change_Year_1522', 'Ref_Change_LCover_1522', 'Def2016', 'Def2017', 'Def2018', 'Def2019', 'Def2020', 'Def2021', 'Def2022', 'Deg2016', 'Deg2017', 'Deg2018', 'Deg2019', 'Deg2020', 'Deg2021', 'Deg2022', 'Defall', 'Degall', 'Stable', 'NF', 'DensFor', 'DensDryFor', 'SecFor', 'DryOpenFor', 'Mangrove', 'Swamp', 'Gallery', 'Plantation', 'Woodland', 'Shrubland', 'Grassland', 'Aquatic', 'Bare', 'Cultivated', 'Builtup', 'Water', 'DensFor_Def', 'DensDryFor_Def', 'SecFor_Def', 'DryOpenFor_Def', 'Mangrove_Def', 'Swamp_Def', 'Gallery_Def', 'Plantation_Def', 'Woodland_Def', 'DensFor_Deg', 'DensDryFor_Deg', 'SecFor_Deg', 'DryOpenFor_Deg', 'Mangrov

In [91]:
len(joined_data)

10329

In [92]:
joined_data = joined_data.to_crs("EPSG:4326")

In [93]:
# if no more data are to be joined, save output
out_file_shp_att =  '/home/sepal-user/module_results/esbae/GAB/GAB_all_ceo_2015_2022_2023_prov.csv'
joined_data.to_csv(out_file_shp_att,index=False)

#### get attributes from local raster

In [ ]:
LC_map = "/home/sepal-user/data/lc_map/CAFI_LC_2015_19_wgs84.tif"

In [ ]:
LC_src = rasterio.open(LC_map)

In [ ]:
joined_data["CAFI_LC_2015"] = [x for x in LC_src.sample(coord_list)]
joined_data["CAFI_LC_2015"] = joined_data["CAFI_LC_2015"].str[0]
joined_data.head()

In [ ]:
joined_data['CAFI_LC_2015'].value_counts(dropna=False)

In [ ]:
joined_columns = joined_data.columns.tolist()
# Display the list of column names
print(joined_columns)

In [ ]:
joined_data.head()

#### get attributes from GEE asset

In [174]:
#first time
gdf = joined_data

In [185]:
#second time
gdf = merged_df

In [186]:
# Assuming df is your DataFrame
if gdf['UID'].nunique() == len(gdf):
    print("UID has entirely unique values.")
else:
    print("UID does not have entirely unique values.")

if gdf['ID'].nunique() == len(gdf):
    print("ID has entirely unique values.")
else:
    print("ID does not have entirely unique values.")

UID has entirely unique values.
ID does not have entirely unique values.


In [187]:
#path to asset
asset = ee.Image('users/faocongo/sbae/EQG_FNF_kmeans_strat_5_2015_2022_v2')
#asset = ee.Image('users/faocongo/sbae/EQG_FNF_kmeans_strat_5_2022_2023_v2')
#asset = ee.Image('users/faocongo/sbae/CAF_FNF_kmeans_strat_5_2015_2022')
#asset = ee.Image('users/faocongo/sbae/CMR_FNF_kmeans_strat_5_2015_2022')
#asset = ee.Image('users/faocongo/sbae/CMR_FNF_kmeans_strat_5_2022_2023')
#asset = ee.Image('users/faocongo/sbae/CAF_FNF_kmeans_strat_5_2022_2023')

In [188]:
#what to rename sampled band to. it is always called first
newName = 'TNT_stratum_1522'
#newName = 'TNT_stratum_2223'

In [189]:
#resolution of asset
scale = 70

In [190]:
#name of band
band = 'strata'

In [191]:
gdf = gdf.reset_index(drop=True)
#with just a few columns
#this can take a little while with many points
dfs = []
for i in range(0, len(points), 5000):
    tmp_df = gdf.loc[i:i+4999]
    tmp_fc = geemap.gdf_to_ee(tmp_df[['UID', 'geometry']])
    tmp_smp = asset.select(band).reduceRegions(**{
        'collection': tmp_fc,
        'scale': scale,
        'reducer': ee.Reducer.first()   
    })

    tmp_res = geemap.ee_to_gdf(tmp_smp)
    dfs.append(tmp_res)
    
sampled_df = pd.concat(dfs)

In [192]:
sampled_df

,geometry,UID,first
0,POINT (8.46065 3.45399),0,3
1,POINT (10.82731 1.75688),1,1
2,POINT (10.7479 1.60262),2,4
3,POINT (10.53471 1.18789),3,3
4,POINT (10.90676 1.93037),4,1
...,...,...,...
230,POINT (5.61173 -1.41345),4671,NaN
231,POINT (5.6306 -1.47319),4674,NaN
232,POINT (11.0089 2.18026),4698,NaN
233,POINT (9.95814 0.96349),4939,2


In [193]:
sampled_df = sampled_df.rename(columns={'first':newName})
sampled_df.head()

,geometry,UID,TNT_stratum_1522
0,POINT (8.46065 3.45399),0,3
1,POINT (10.82731 1.75688),1,1
2,POINT (10.7479 1.60262),2,4
3,POINT (10.53471 1.18789),3,3
4,POINT (10.90676 1.93037),4,1


In [194]:
# recover columns
# Step 1: Merge the two DataFrames on the 'ID' column (assuming 'ID' is the common key)
merged_df = pd.merge(sampled_df, gdf, on='UID', how='left', suffixes=('', '_dup'))

# Step 2: Remove duplicate columns
# Here, we assume that any duplicate column from df2 will have the '_dup' suffix added by the merge
# We will keep the columns from df1 and drop the duplicate ones from df2.
merged_df = merged_df.loc[:, ~merged_df.columns.str.endswith('_dup')]
merged_df.head()

,geometry,UID,TNT_stratum_1522,TNT_stratum_2223,ID,plotid,point_id,sampleid,lon,lat,...,Ref_Ftype_2022,Ref_LCover_2022,Ref_Change_Year_2023,Ref_Change_LCover_2023,Def2023,Deg2023,interpreted_qc,Ref_Change_Type_1523,Ref_Change_Year_1523,Province
0,POINT (8.46065 3.45399),0,3,1,0,45,45,45,8,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stable,Stable,Bioko Sur
1,POINT (10.82731 1.75688),1,1,4,1,15952,15952,15952,11,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NF,NF,Wele-Nzas Province
2,POINT (10.7479 1.60262),2,4,2,2,15936,15936,15936,11,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stable,Stable,Wele-Nzas Province
3,POINT (10.53471 1.18789),3,3,4,3,15893,15893,15893,11,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deg,Deg2022,Centro Sur Province
4,POINT (10.90676 1.93037),4,1,1,4,15847,15847,15847,11,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stable,Stable,Wele-Nzas Province


In [195]:
merged_df_cols = merged_df.columns.tolist()
# Display the list of column names
print(merged_df_cols)

['geometry', 'UID', 'TNT_stratum_1522', 'TNT_stratum_2223', 'ID', 'plotid', 'point_id', 'sampleid', 'lon', 'lat', 'sample_geom', 'Ref_FNF_2015', 'Ref_Regeneration', 'Ref_Change_1522', 'Ref_Change_Type_1522', 'ArtFor', 'ArtMine', 'InfraR', 'Urb', 'IndFor', 'Other', 'Other_Desc', 'IndMine', 'IndAg', 'ArtAg', 'Ref_Year_1522', 'Ref_NFtype_2015', 'Ref_Ftype_2015', 'collection', 'interpreted', 'Ref_LCover_2015', 'Ref_Change_Year_1522', 'Ref_Change_LCover_1522', 'Def2016', 'Def2017', 'Def2018', 'Def2019', 'Def2020', 'Def2021', 'Def2022', 'Deg2016', 'Deg2017', 'Deg2018', 'Deg2019', 'Deg2020', 'Deg2021', 'Deg2022', 'Defall', 'Degall', 'Stable', 'NF', 'DensFor', 'DensDryFor', 'SecFor', 'DryOpenFor', 'Mangrove', 'Swamp', 'Gallery', 'Plantation', 'Woodland', 'Shrubland', 'Grassland', 'Aquatic', 'Bare', 'Cultivated', 'Builtup', 'Water', 'DensFor_Def', 'DensDryFor_Def', 'SecFor_Def', 'DryOpenFor_Def', 'Mangrove_Def', 'Swamp_Def', 'Gallery_Def', 'Plantation_Def', 'Woodland_Def', 'DensFor_Deg', 'DensD

In [196]:
#drop any columns if needed
columns_to_drop = ['sample_geom']
#joined_data = joined_data.drop(columns_to_drop, axis=1)

##### repeat above as needed

In [197]:
pd.pivot_table(
    merged_df,
    values='UID',
    index=['TNT_stratum_1522'],
    columns=['collection'],
    aggfunc="count",
    margins=True,              # Adds totals
    margins_name='Total',      # Name of the margins/total column
    dropna=False               # Include NaN values in the table
)

collection,coll_1522_1,coll_1522_2,coll_2023_1,coll_2023_2,coll_2023_3,Total
TNT_stratum_1522,,,,,,
0,8,58,3,4,NaN,73
1,158,1043,87,149,6,1443
2,146,774,53,139,2,1114
3,96,790,56,115,1,1058
4,56,761,26,62,NaN,905
5,29,541,13,28,NaN,611
NaN,5,17,9,NaN,NaN,NaN
Total,498,3984,247,497,9,5235


In [198]:
pd.pivot_table(
    merged_df,
    values='UID',
    index=['TNT_stratum_2223'],
    columns=['collection'],
    aggfunc="count",
    margins=True,              # Adds totals
    margins_name='Total',      # Name of the margins/total column
    dropna=False               # Include NaN values in the table
)

collection,coll_1522_1,coll_1522_2,coll_2023_1,coll_2023_2,coll_2023_3,Total
TNT_stratum_2223,,,,,,
0,1,5,NaN,NaN,NaN,6
1,269,2197,109,236,NaN,2811
2,58,379,21,59,NaN,517
3,76,575,52,76,NaN,779
4,74,643,45,97,3,862
5,15,168,11,29,6,229
NaN,5,17,9,NaN,NaN,NaN
Total,498,3984,247,497,9,5235


In [199]:
# Remove rows where the stratum has NaN values - outside the country
merged_df = merged_df.dropna(subset=['TNT_stratum_2223'])

In [200]:
# Check if the lengths of merged_df and ceo are the same
if len(merged_df) == len(points):
    print("The lengths of merged_df and ceo are the same.")
else:
    print("The lengths of merged_df and ceo are different.")

# Optionally, you can print the lengths for clarity
print("Length of merged_df:", len(merged_df))
print("Length of ceo:", len(points))

The lengths of merged_df and ceo are different.
Length of merged_df: 5204
Length of ceo: 5235


#### export

In [201]:
out_file =  '/home/sepal-user/module_results/esbae/EQG/EQG_all_ceo_2015_2022_2023_TNTstrat.csv'
#sampled_df.to_csv(out_file,index=False)
merged_df.to_csv(out_file,index=False)